In [7]:
# 13:45
# 14:05

import random
import numpy as np
import math


def genDat(n,l):
    rotL = lambda x: x[1:] + [x[0]]
    rand = lambda y: map(lambda z:random.choice([0,1]), [0]*y)
    
    x = map(rand, [l]*n)
    y = map(rotL, x)
    
    x = map(np.array, x)
    y = map(np.array, y)
    
    return zip(x,y)

def logsigmoid(z):
    return 1.0/(1 + math.exp(-z))

def dlogsigmoid(z):
    f = logsigmoid
    
    return f(z) * (1 - f(z))


class NN:
    def __init__(_,ll,lr=0.05):
        _.ll  = ll
        _.lr  = lr
        _.fA  = np.vectorize(logsigmoid)
        _.dfA = np.vectorize(dlogsigmoid)
        _.w, _.b = _.genWB()
        
    def genWB(_):
        w = [np.random.randn(nin, nout) for (nin, nout) in zip(_.ll[:-1], _.ll[1:])]
        b = [np.random.randn(nout)      for nout        in  _.ll[1:]]
        
        return w, b
    
    def ff(_,x):
        v  = [None] * len(_.ll)
        vf = [None] * len(_.ll)
        
        vf[0] = x
        v [0] = x
        
        for i in range(len(_.w)):
            v[i+1]   = np.dot(np.transpose(_.w[i]), vf[i]) + _.b[i] 
            vf[i+1]  = _.fA(v[i+1])
            
        return v,vf
    
    def ff_pure(_,x):
        v = x
        for i in range(len(_.w)):
            v   = _.fA( np.dot(np.transpose(_.w[i]), v) + _.b[i] )
        return v
    
    
    def fb(_,v,vf,y):
        g  = [None] * len(_.w)
        gw = [None] * len(_.w)
        
        g[-1] = (vf[-1] - y) * _.dfA(v[-1])
        
        for i in range(len(_.w)-2,-1,-1):
            g[i]  = np.multiply(np.dot(_.w[i+1], g[i+1]), _.dfA(v[i+1]))
            
        for i in range(len(_.w)-1,-1,-1):
            gw[i] = np.transpose(np.outer(g[i], vf[i]) )
            
        return gw,g
    
    def trainStep(_,x,y):
        v,vf  = _.ff(x)
        gw,gb = _.fb(v,vf,y)
        
        _.w = _.w - np.multiply(gw, _.lr)
        _.b = _.b - np.multiply(gb, _.lr)
        
        out = _.ff_pure(x)
        (e1,e2) = (_.err(vf[-1], y), _.err(out, y))
        return (e1,e2)
            
        
    def err(_, out, y):
        return math.sqrt(sum((out - y) ** 2))
    
    def train(_,data):
        return [_.trainStep(x,y) for (x,y) in data]
    
    def test(_,data):
        for (x,y) in data:
            out = _.ff_pure(x)
            e   = _.err(out,y)
            print "x = {}, y = {}, out = {}, e = {}".format(x,y,out,e)
            
dat = genDat(30000,3)

layers = [[3,3,3], [3,4,4,3]]

for ll in layers:
    print "ll = {}".format(ll)
    nn = NN(ll)
    nn.train(dat)
    nn.test(dat[:5])
    
        

ll = [3, 3, 3]
x = [1 1 1], y = [1 1 1], out = [ 0.94182185  0.9101905   0.97093182], e = 0.110884638681
x = [1 0 1], y = [0 1 1], out = [ 0.05544412  0.95232486  0.97137152], e = 0.0785274442327
x = [0 1 1], y = [1 1 0], out = [ 0.94504816  0.91271323  0.065227  ], e = 0.122037888969
x = [1 1 0], y = [1 0 1], out = [ 0.89828368  0.10153631  0.92660323], e = 0.161378186483
x = [0 0 1], y = [0 1 0], out = [ 0.07897739  0.92889899  0.06222903], e = 0.123147202258
ll = [3, 4, 4, 3]
x = [1 1 1], y = [1 1 1], out = [ 0.91219363  0.93443491  0.93913666], e = 0.125351844849
x = [1 0 1], y = [0 1 1], out = [ 0.03764153  0.92456792  0.92954871], e = 0.109864771051
x = [0 1 1], y = [1 1 0], out = [ 0.92830107  0.92774137  0.04928523], e = 0.113097656794
x = [1 1 0], y = [1 0 1], out = [ 0.94864609  0.04757797  0.94835766], e = 0.0869932139768
x = [0 0 1], y = [0 1 0], out = [ 0.0511487   0.95083787  0.03482239], e = 0.0790297626069
